In [1]:
import os
import torch
import argparse
import itertools
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import get_rank, init_process_group, destroy_process_group, all_gather, get_world_size
from torch import Tensor
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from glob import glob
from torch.utils.data.distributed import DistributedSampler
import random
from conditionDiffusion.unet import Unet
from conditionDiffusion.embedding import ConditionalEmbedding
from conditionDiffusion.utils import get_named_beta_schedule
from conditionDiffusion.diffusion import GaussianDiffusion
from conditionDiffusion.Scheduler import GradualWarmupScheduler
from PIL import Image
import styleGAN.networks_stylegan as stylegan
print(f"GPUs used:\t{torch.cuda.device_count()}")
device = torch.device("cuda",4)
print(f"Device:\t\t{device}")
import pytorch_model_summary as tms

/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/torch/onnx/_internal/_beartype.py:36: UserWarning: unhashable type: 'list'
  warnings.warn(f"{e}")


GPUs used:	8
Device:		cuda:4


In [2]:
class_list=['유형1','유형2']
params={'image_size':1024,
        'lr':1e-5,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':1,
        'epochs':1000,
        'n_classes':None,
        'data_path':'../../data/origin_type/STNT/',
        'image_count':5000,
        'inch':3,
        'modch':64,
        'outch':3,
        'chmul':[1,2,4,8,16,16,16],
        'numres':2,
        'dtype':torch.float32,
        'cdim':10,
        'useconv':False,
        'droprate':0.1,
        'T':1000,
        'w':1.8,
        'v':0.3,
        'multiplier':2.5,
        'threshold':0.1,
        'ddim':True,
        }


In [3]:
trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

def transback(data:Tensor) -> Tensor:
    return data / 2 + 0.5

class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self,parmas, images,label):
        
        self.images = images
        self.args=parmas
        self.label=label
        
    def trans(self,image):
        if random.random() > 0.5:
            transform = transforms.RandomHorizontalFlip(1)
            image = transform(image)
            
        if random.random() > 0.5:
            transform = transforms.RandomVerticalFlip(1)
            image = transform(image)
            
        return image
    
    def __getitem__(self, index):
        image=self.images[index]
        label=self.label[index]
        image = self.trans(image)
        return image,label
    
    def __len__(self):
        return len(self.images)


image_label=[]
image_path=[]
for i in tqdm(range(len(class_list))):
    image_list=glob(params['data_path']+class_list[i]+'/*.jpeg')
    for j in range(len(image_list)):
        image_path.append(image_list[j])
        image_label.append(i)
        
train_images=torch.zeros((len(image_path),params['inch'],params['image_size'],params['image_size']))
for i in tqdm(range(len(image_path))):
    train_images[i]=trans(Image.open(image_path[i]).convert('RGB').resize((params['image_size'],params['image_size'])))
train_dataset=CustomDataset(params,train_images,image_label)
dataloader=DataLoader(train_dataset,batch_size=params['batch_size'],shuffle=True)

100%|██████████| 2/2 [00:00<00:00, 24.07it/s]


KeyboardInterrupt: 

In [4]:
generator = stylegan.Generator(
    z_dim=512,              # 잠재 벡터 Z의 차원
    c_dim=10,               # 조건 레이블 C의 차원 (10개의 클래스)
    w_dim=512,              # 중간 잠재 벡터 W의 차원
    img_resolution=512,     # 출력 이미지 해상도 (512x512)
    img_channels=3,         # 출력 이미지의 채널 수 (RGB 이미지)
    num_classes=10,         # 데이터셋의 클래스 개수
    mapping_kwargs={"lr_multiplier": 0.01},  # MappingNetwork에 전달할 추가 인자 
)
z = torch.randn(1, 512)  # 잠재 벡터
y = torch.randint(0, 10, (1,))  # 임의의 레이블

# 모델 구조를 출력합니다.
print(tms.summary(generator, torch.zeros(1, 512), torch.zeros(1, dtype=torch.int64), show_input=False))

Input Layer Output Size: torch.Size([1, 512, 36, 36])
Layer L0_36_512 Output Size: torch.Size([1, 512, 55, 55])
Layer L1_36_512 Output Size: torch.Size([1, 512, 74, 74])
Layer L2_52_512 Output Size: torch.Size([1, 512, 61, 61])
Layer L3_52_512 Output Size: torch.Size([1, 512, 80, 80])
Layer L4_84_512 Output Size: torch.Size([1, 512, 67, 67])
Layer L5_84_512 Output Size: torch.Size([1, 512, 86, 86])
Layer L6_148_512 Output Size: torch.Size([1, 512, 73, 73])
Layer L7_148_512 Output Size: torch.Size([1, 512, 92, 92])
Layer L8_276_512 Output Size: torch.Size([1, 512, 79, 79])
Layer L9_276_431 Output Size: torch.Size([1, 431, 98, 98])
Layer L10_532_287 Output Size: torch.Size([1, 287, 85, 85])
Layer L11_532_192 Output Size: torch.Size([1, 192, 104, 104])
Layer L12_532_128 Output Size: torch.Size([1, 128, 123, 123])
Layer L13_512_128 Output Size: torch.Size([1, 128, 102, 102])
Layer L14_512_3 Output Size: torch.Size([1, 3, 102, 102])
Final Output Size: torch.Size([1, 3, 102, 102])
----------

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SynthesisNetwork(nn.Module):
    def __init__(self, w_dim, img_resolution, img_channels):
        super().__init__()
        self.w_dim = w_dim
        self.img_resolution = img_resolution
        self.img_channels = img_channels

        # Define the layers based on the resolution
        self.num_layers = int(np.log2(img_resolution)) * 2 - 2
        self.channels = {4: 512, 8: 512, 16: 512, 32: 512, 64: 256, 128: 128, 256: 64, 512: 32, 1024: 16}

        self.input_layer = nn.Parameter(torch.randn(1, self.channels[4], 4, 4))
        self.conv_layers = nn.ModuleList()
        self.to_rgb_layers = nn.ModuleList()

        for res in range(3, self.num_layers + 3):  # 수정된 부분: 범위를 3부터 시작
            in_channels = self.channels[2 ** (res - 2)]
            out_channels = self.channels[2 ** (res - 1)]
            self.conv_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
            self.conv_layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
            self.to_rgb_layers.append(nn.Conv2d(out_channels, img_channels, kernel_size=1))

    def forward(self, w):
        x = self.input_layer.repeat(w.shape[0], 1, 1, 1)
        for i in range(0, len(self.conv_layers), 2):
            x = F.leaky_relu(self.conv_layers[i](x), 0.2)
            x = F.leaky_relu(self.conv_layers[i + 1](x), 0.2)
            if i // 2 < len(self.to_rgb_layers):
                rgb = self.to_rgb_layers[i // 2](x)
                if i == 0:
                    img = rgb
                else:
                    img = F.interpolate(img, scale_factor=2, mode='bilinear', align_corners=False)
                    img = img + rgb
        return img

# Example usage
w_dim = 512
img_resolution = 512
img_channels = 3

synthesis_network = SynthesisNetwork(w_dim, img_resolution, img_channels)
w = torch.randn(1, 16, 512)  # Example latent vector
output_img = synthesis_network(w)
print(output_img.shape)  # Should be [1, 3, 512, 512]

KeyError: 2